# Generic Chains Overview

## Simple Chain

<font color='green'>
The most elementary type of chain is known as a basic chain, which represents the simplest form of crafting a chain. <br>In this setup, there is only one Language Model (LLM) responsible for receiving an input prompt and using it for generating text.
<font>

In [142]:
#Please install the package as per your requirement :)
#!pip install openai==1.10.0
#!pip install langchain==0.1.4
#!pip install huggingface-hub==0.20.3
#!pip install langchain-openai==0.0.5

In [143]:
from dotenv import load_dotenv
load_dotenv()

True

In [144]:
#The below import has been replaced by the later one
#from langchain.llms import OpenAI
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [145]:
llm = OpenAI()

In [146]:
prompt = PromptTemplate(
    input_variables=["place"],
    template="Best places to visit in {place}?",
)

In [147]:
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
# Recently langchain has replaced 'run' function with 'invoke'
print(chain.invoke("India"))

{'place': 'India', 'text': "\n\n1. The Taj Mahal, Agra\n2. The Golden Temple, Amritsar\n3. Jaipur, the Pink City\n4. Goa's beaches and nightlife\n5. Kerala's backwaters\n6. Varanasi, the holy city\n7. Ranthambore National Park for tiger sightings\n8. Ladakh for its stunning landscapes\n9. Hampi's ancient ruins\n10. Jaisalmer's desert and forts\n11. Darjeeling for tea plantations and trekking\n12. Mumbai for its bustling city life and Bollywood culture\n13. Udaipur's romantic lakes and palaces\n14. Andaman and Nicobar Islands for its beautiful beaches and marine life\n15. Mysore for its royal heritage and temples."}


In [148]:
chain.invoke("India")

{'place': 'India',
 'text': '\n\n1. The Taj Mahal, Agra\n2. Varanasi, Uttar Pradesh\n3. Jaipur, Rajasthan\n4. Goa\n5. Kerala backwaters\n6. Ladakh\n7. Hampi, Karnataka\n8. Jaisalmer, Rajasthan\n9. Darjeeling, West Bengal\n10. Rishikesh, Uttarakhand\n11. Mumbai, Maharashtra\n12. Delhi\n13. Udaipur, Rajasthan\n14. Amritsar, Punjab\n15. Mysore, Karnataka\n16. Shimla, Himachal Pradesh\n17. Khajuraho, Madhya Pradesh\n18. Pushkar, Rajasthan\n19. Alleppey, Kerala\n20. Jodhpur, Rajasthan'}

## Simple Sequential Chains

<font color='green'>
Sequential Chains involves making a series of consecutive calls to the language model.<br> This approach proves especially valuable when there is a need to utilize the output generated from one call as the input for another call.
<font>

In [149]:
from langchain.chains import SimpleSequentialChain
from langchain import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint

In [150]:
template = """You have to suggest 5 best places names with out details to visit in {place}? 

YOUR RESPONSE: 
"""
# PromptTemplate(
#     input_variables=["place"],
#     template="Best places to visit in {place}?",
# )
prompt_template = PromptTemplate(
    input_variables=["place"], 
    template=template)

In [151]:
# HF_llm= HuggingFaceHub(repo_id = "google/flan-t5-large")
HF_llm= HuggingFaceEndpoint(repo_id = "google/flan-t5-xxl")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.


Token is valid (permission: read).
Your token has been saved to C:\Users\jdamodhar\.cache\huggingface\token
Login successful


In [152]:
#llm = OpenAI()

In [153]:
place_chain = LLMChain(llm=HF_llm, prompt=prompt_template)

In [154]:
template = """Given a list a places, please estimate the expenses to visit all of them in local currency and also the days needed
{expenses}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(
    input_variables=["expenses"],
    template=template)

In [155]:
expenses_chain = LLMChain(llm=llm, prompt=prompt_template)

In [156]:
final_chain = SimpleSequentialChain(chains=[place_chain, expenses_chain], verbose=True)

In [157]:
review = final_chain.invoke("India")



> Entering new SimpleSequentialChain chain...


shimla, shimla, india, dharamshala

Based on research and average prices, the estimated expenses for visiting all of the places would be around 25,000 Indian Rupees (INR). This includes accommodation, transportation, food, and entrance fees. The estimated number of days needed to visit all the places would be 6-7 days. This would allow for enough time to explore each place and also account for travel time between the locations. Please note that these estimates may vary depending on personal preferences and travel styles.

> Finished chain.


In [163]:
HF_llm= HuggingFaceHub(repo_id = "google/flan-t5-xxl")
# HF_llm= HuggingFaceEndpoint(repo_id = "google/flan-t5-large")
#llm = OpenAI()
template = """You have to suggest 5 best places names to visit in {place}? 

YOUR RESPONSE: 
"""
# PromptTemplate(
#     input_variables=["place"],
#     template="Best places to visit in {place}?",
# )
prompt_template = PromptTemplate(
    input_variables=["place"], 
    template=template)
place_chain = LLMChain(llm=HF_llm, prompt=prompt_template)
template = """Given a list a places, please estimate the expenses to visit all of them in local currency and also the days needed
{expenses}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(
    input_variables=["expenses"],
    template=template)
llm = OpenAI()
expenses_chain = LLMChain(llm=llm, prompt=prompt_template)
final_chain = SimpleSequentialChain(chains=[place_chain, expenses_chain], verbose=True)
review = final_chain.invoke("India")



> Entering new SimpleSequentialChain chain...
-Mumbai -Agra -Madhya Pradesh -Kerala

Mumbai:
- Transportation (round trip flight from major international airport): INR 20,000
- Accommodation (3 nights in a mid-range hotel): INR 10,000
- Food and drinks: INR 5,000
- Sightseeing and activities: INR 5,000
Total estimated expenses for Mumbai: INR 40,000
Estimated days needed for Mumbai: 3 days

Agra:
- Transportation (train or bus from Mumbai): INR 2,000
- Accommodation (1 night in a mid-range hotel): INR 3,000
- Food and drinks: INR 1,500
- Sightseeing and activities: INR 2,500
Total estimated expenses for Agra: INR 9,000
Estimated days needed for Agra: 1 day

Madhya Pradesh:
- Transportation (train or bus from Agra): INR 5,000
- Accommodation (3 nights in a mid-range hotel): INR 9,000
- Food and drinks: INR 4,500
- Sightseeing and activities: INR 5,000
Total estimated expenses for Madhya Pradesh: INR 23,500

> Finished chain.


In [161]:
HF_llm= HuggingFaceHub(repo_id = "google/flan-t5-xxl")
# HF_llm= HuggingFaceEndpoint(repo_id = "google/flan-t5-large")
#llm = OpenAI()
template = """You have to suggest 5 best places names with out details to visit in {place}? 

YOUR RESPONSE: 
"""
# PromptTemplate(
#     input_variables=["place"],
#     template="Best places to visit in {place}?",
# )
prompt_template = PromptTemplate(
    input_variables=["place"], 
    template=template)
place_chain = LLMChain(llm=llm, prompt=prompt_template)
template = """Given a list a places, please estimate the expenses to visit all of them in local currency and also the days needed
{expenses}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(
    input_variables=["expenses"],
    template=template)
llm = OpenAI()
expenses_chain = LLMChain(llm=llm, prompt=prompt_template)
final_chain = SimpleSequentialChain(chains=[place_chain, expenses_chain], verbose=True)
review = final_chain.invoke("India")



> Entering new SimpleSequentialChain chain...
1. Goa
2. Jaipur
3. Manali
4. Agra
5. Udaipur
1. Goa
Expenses: Approximately 20,000 INR for 3-4 days
Days Needed: 3-4 days

2. Jaipur
Expenses: Approximately 15,000 INR for 2-3 days
Days Needed: 2-3 days

3. Manali
Expenses: Approximately 25,000 INR for 4-5 days
Days Needed: 4-5 days

4. Agra
Expenses: Approximately 10,000 INR for 1-2 days
Days Needed: 1-2 days

5. Udaipur
Expenses: Approximately 15,000 INR for 2-3 days
Days Needed: 2-3 days

> Finished chain.
